#1. Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
cust=pd.read_excel('Dataset.xlsx',sheet_name='CustomerList')

#2. Data Quality Analysis

2.1 Checking records

In [ ]:
cust.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.55,0.6875,0.859375,0.730469,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.47,0.4700,0.587500,0.499375,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,0.57,0.5700,0.570000,0.570000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.59,0.7375,0.737500,0.737500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.77,0.7700,0.962500,0.962500,4,4,1.703125


2.2. Check information of columns and their datatypes

In [ ]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   int64  
 4   DOB                                  983 non-null    object 
 5   job_title                            894 non-null    object 
 6   job_industry_category                835 non-null    object 
 7   wealth_segment                       1000 non-null   object 
 8   deceased_indicator                   1000 non-null   object 
 9   owns_car                             1000 non-null   object 
 10  tenure                               1000 non-null   int64  
 11  address                        

column 16-20 are unnamed so they should be dropped

#1. Dropping column

In [ ]:
cust.drop(labels=['Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20'], axis=1 , inplace=True)

#2. Missing Value

In [ ]:

# Total number of missing values
cust.isnull().sum()

first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
dtype: int64

last_name                               29
DOB                                     17
job_title                              106

2.1 ***last_name***

a Customer can have only first name to be identified. If last name is missing it is not a matter of concern . So we replace llast name with none.
cust[new_cust['last_name'].isnull()]: This part uses the boolean Series from the previous step to filter the ust DataFrame,returning only the rows where the 'last_name' is NaN.

In [ ]:

#count of NaN values in the 'first_name' column of rows
#where the 'last_name' is also NaN in the new_cust DataFrame
cust[cust['last_name'].isnull()]['first_name'].isnull().sum()

0

In [ ]:
cust['last_name'].fillna('None',axis=0, inplace=True)


In [ ]:
cust['last_name'].isnull().sum()

0

2.2 DOB

In [ ]:
cust["DOB"].isnull().mean()*100

1.7000000000000002

Since null values are 1.7% we can drop those rows where dob is null

In [ ]:
dob_to_drop=cust[cust['DOB'].isnull()].index

In [ ]:
cust.drop(index=dob_to_drop,inplace=True,axis=0)

In [ ]:
cust['DOB'].isnull().sum()

0

There can be case where age can be invalid,but we do not have age column so we create a function age to find out current age and remove any invalid age if found.

In [ ]:
cust['DOB'].info()

<class 'pandas.core.series.Series'>
Index: 983 entries, 0 to 999
Series name: DOB
Non-Null Count  Dtype 
--------------  ----- 
983 non-null    object
dtypes: object(1)
memory usage: 15.4+ KB


In [ ]:
from datetime import datetime

def calc_age(born):
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

#There was an error in DOB column where the column DOB contails string and not datetime objects.
#So we converted DOB to datetime format
#5 DOB 3913 non-null   datetime64[ns]
cust['DOB'] = pd.to_datetime(cust['DOB'])

#create a new column named age and feeding it with calc_age
cust['Age'] = cust['DOB'].apply(calc_age)

#later you can check DOB datatype changes from **object** to **datetime64[ns]**

In [ ]:
cust['Age'].describe()

count    983.000000
mean      52.517803
std       17.045388
min       22.000000
25%       40.500000
50%       52.000000
75%       66.000000
max       85.000000
Name: Age, dtype: float64

During Subtask find out: a) age group chart b) age group of customers that are newly associated

In [ ]:
#creating age-group
import math
cust['Age_Group'] = cust['Age'].apply(lambda x : (math.floor(x/10)+1)*10)

2.3 Job Title

In [ ]:
cust[cust['job_title'].isnull()]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,Age,Age_Group
15,Dukie,Swire,Male,88,1954-03-31,NaN,Manufacturing,Affluent Customer,N,Yes,5,64 Granby Parkway,2500,NSW,Australia,8,16,1.562500,70,80
25,Rourke,Gillbard,Male,11,1945-08-03,NaN,Property,Mass Customer,N,No,17,75 Cordelia Trail,4817,QLD,Australia,4,26,1.468750,78,80
29,Rhona,De Freyne,Female,45,1960-11-22,NaN,Health,High Net Worth,N,No,8,11184 East Drive,3056,VIC,Australia,10,30,1.460938,63,70
30,Sharron,Claibourn,Female,62,1980-01-26,NaN,Financial Services,High Net Worth,N,Yes,17,555 Hermina Avenue,2280,NSW,Australia,8,30,1.460938,44,50
37,Mitchell,MacCague,Male,58,1979-04-11,NaN,Manufacturing,Mass Customer,N,No,15,240 Acker Avenue,3190,VIC,Australia,8,38,1.437500,45,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,Candy,None,Female,23,1977-12-08,NaN,Financial Services,Mass Customer,N,No,6,59252 Maryland Drive,3500,VIC,Australia,3,951,0.450500,46,50
953,Noami,Cokly,Female,74,1962-09-17,NaN,Manufacturing,Mass Customer,N,Yes,15,2886 Buena Vista Terrace,2038,NSW,Australia,11,954,0.450000,61,70
971,Frieda,Tavinor,Female,43,1999-03-04,NaN,NaN,Affluent Customer,N,No,10,7 Mallory Lane,3064,VIC,Australia,6,972,0.430000,25,30
972,Ellwood,Budden,Male,82,1998-06-03,NaN,Health,Mass Customer,N,Yes,11,79907 Randy Center,2192,NSW,Australia,10,972,0.430000,25,30


In [ ]:
cust['job_title'].isnull().mean()*100

10.681586978636826

In [ ]:
cust['job_title'].fillna('missing', inplace=True, axis=0)

In [ ]:
cust['job_title'].isnull().sum()

0

2.4 Job Industry Category

In [ ]:
cust['job_industry_category'].isnull().mean()*100

16.7853509664293

Job industry category is 16.7 so we replace the values with misisng

In [ ]:
cust['job_industry_category'].fillna('missing', inplace=True, axis=0)

In [ ]:

cust['job_industry_category'].isnull().sum()

0

In [ ]:
#Rechecking missing value
cust.isnull().sum()

first_name                             0
last_name                              0
gender                                 0
past_3_years_bike_related_purchases    0
DOB                                    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
deceased_indicator                     0
owns_car                               0
tenure                                 0
address                                0
postcode                               0
state                                  0
country                                0
property_valuation                     0
Rank                                   0
Value                                  0
Age                                    0
Age_Group                              0
dtype: int64

#3. Data Inconsistency

To check inconsistency in categorical column

3.1 Gender

In [ ]:
cust['gender'].value_counts()

gender
Female    513
Male      470
Name: count, dtype: int64

3.2 wealth segment

In [ ]:
cust['wealth_segment'].value_counts()

wealth_segment
Mass Customer        499
High Net Worth       249
Affluent Customer    235
Name: count, dtype: int64

3.3. deceased indicator

In [ ]:
cust['deceased_indicator'].value_counts()

deceased_indicator
N    983
Name: count, dtype: int64

In [ ]:
cust['owns_car'].value_counts()

owns_car
No     497
Yes    486
Name: count, dtype: int64

In [ ]:
cust['state'].value_counts()

state
NSW    499
VIC    258
QLD    226
Name: count, dtype: int64

In [ ]:
cust['country'].value_counts()

country
Australia    983
Name: count, dtype: int64

In [ ]:
#To find and display unique combinations of values in the 'postcode' and 'state' columns in the cust
cust[['postcode', 'state']].drop_duplicates().sort_values('state')

,postcode,state
164,2073,NSW
202,2300,NSW
616,2049,NSW
204,2429,NSW
615,2070,NSW
...,...,...
234,3429,VIC
589,3037,VIC
604,3129,VIC
245,3134,VIC


3.8 Address

In [ ]:
cust[['address', 'postcode','state','country']].sort_values('address')

,address,postcode,state,country
721,0 Bay Drive,2750,NSW,Australia
138,0 Dexter Parkway,2380,NSW,Australia
624,0 Emmet Trail,4128,QLD,Australia
300,0 Esker Avenue,4019,QLD,Australia
685,0 Express Lane,2142,NSW,Australia
...,...,...,...,...
308,99 Westend Court,2287,NSW,Australia
336,990 Hoffman Avenue,3029,VIC,Australia
796,99376 Namekagon Street,3101,VIC,Australia
583,9940 Manley Drive,2574,NSW,Australia


3.9 Tenure

To find inconsitency in tenure one must check missing values, negative value or if the data doesnt fall in range

In [ ]:
#cust['tenure'].value_counts()
cust['tenure'].describe()

count    983.000000
mean      11.459817
std        5.006123
min        1.000000
25%        8.000000
50%       11.000000
75%       15.000000
max       22.000000
Name: tenure, dtype: float64

#4. Duplicate

In [ ]:
duplicated_values=cust.drop_duplicates()
print(duplicated_values.shape[0])
print(cust.shape[0])


983
983


# 5.Export

In [ ]:
cust.to_csv('CustomerList_Cleaned.csv', index=False)